In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Groupby한 Series에서 4, 5 act_label 널값 처리를 못 하고 있다.

act = pd.read_csv('stackPerDevice.csv')
act.shape


(71957, 9)

In [98]:
act = act[ act['deviceId'] == 'id_MzQ6QUI6OTU6NzE6QTQ6Mjg']
act

,deviceId,activity_key,avg_presence,avg_people_cnt,avg_activity,max_activity,cnt,first_eventtime,lastStatus
61259,id_MzQ6QUI6OTU6NzE6QTQ6Mjg,2769933,0,0.0,4.830583,14.8982,6,NaN,NaN
61260,id_MzQ6QUI6OTU6NzE6QTQ6Mjg,2769937,0,0.0,5.206659,22.1080,54,2022-09-01 01:16:03.771,fall_exit
61261,id_MzQ6QUI6OTU6NzE6QTQ6Mjg,2769938,0,0.0,2.848062,16.0897,21,2022-09-01 01:20:08.681,fall_exit
61262,id_MzQ6QUI6OTU6NzE6QTQ6Mjg,2769940,0,0.0,5.868583,34.3912,42,2022-09-01 01:45:46.251,fall_exit
61263,id_MzQ6QUI6OTU6NzE6QTQ6Mjg,2769944,0,0.0,0.090550,0.0942,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...
63513,id_MzQ6QUI6OTU6NzE6QTQ6Mjg,2774242,0,0.0,8.335829,48.1720,14,NaN,NaN
63514,id_MzQ6QUI6OTU6NzE6QTQ6Mjg,2774243,0,0.0,1.285760,2.6238,5,NaN,NaN
63515,id_MzQ6QUI6OTU6NzE6QTQ6Mjg,2774244,0,0.0,3.931575,14.7158,8,NaN,NaN
63516,id_MzQ6QUI6OTU6NzE6QTQ6Mjg,2774245,0,0.0,4.870035,32.7002,96,2022-09-30 23:10:36.728,fall_exit


In [99]:
act = act.drop(['deviceId'], axis=1)
act

,activity_key,avg_presence,avg_people_cnt,avg_activity,max_activity,cnt,first_eventtime,lastStatus
61259,2769933,0,0.0,4.830583,14.8982,6,NaN,NaN
61260,2769937,0,0.0,5.206659,22.1080,54,2022-09-01 01:16:03.771,fall_exit
61261,2769938,0,0.0,2.848062,16.0897,21,2022-09-01 01:20:08.681,fall_exit
61262,2769940,0,0.0,5.868583,34.3912,42,2022-09-01 01:45:46.251,fall_exit
61263,2769944,0,0.0,0.090550,0.0942,2,NaN,NaN
...,...,...,...,...,...,...,...,...
63513,2774242,0,0.0,8.335829,48.1720,14,NaN,NaN
63514,2774243,0,0.0,1.285760,2.6238,5,NaN,NaN
63515,2774244,0,0.0,3.931575,14.7158,8,NaN,NaN
63516,2774245,0,0.0,4.870035,32.7002,96,2022-09-30 23:10:36.728,fall_exit


In [100]:
ABSENCE = 0 # 부재
NOT_ACT = 1 # 미활동
ACT = 2 # 활동
CAUTION = 3 #주의
FALL = 4 # 낙상


def getEmergencyLabel(status, defaultStatus):
    if status == "fall_detected" or status == "fall_exit" or status == "fall_confirmed":
        return CAUTION
    elif status == "calling" or status == "finished":
        return FALL
    else:
        return defaultStatus

def makeStatus(df):
#     if df['avg_people_cnt'] == 0: #없는 경우가 있다.
#         return ABSENCE
#     else:
#     act_label = ABSENCE
    if df['avg_activity'] == 0:
        act_label = ABSENCE
    elif df['avg_activity'] < 3.0 or df['max_activity'] < 20.0:
        act_label = NOT_ACT
    else:
        act_label = ACT
        
    act_label = getEmergencyLabel(df['lastStatus'], act_label)
    return act_label

In [118]:
# FROM_UNIXTIME( presenceTime * 600, '%y/%m/%d') AS DATE1,

from datetime import datetime

ts = datetime.fromtimestamp(2769937*600)
print(ts)
print(ts.strftime('%m-%d'))

2022-09-01 01:10:00
09-01


In [119]:
act[['activity_key']]

,activity_key
61259,2769933
61260,2769937
61261,2769938
61262,2769940
61263,2769944
...,...
63513,2774242
63514,2774243
63515,2774244
63516,2774245


In [120]:
# #디바이스별로 집계를 구하고, 그 결과를 newdata에 저장함.
# newdata = pd.DataFrame()

# # group의 키 목록을 구함
# # grouped = act.groupby('deviceId')

# # for deviceid in grouped.groups.keys():
# #     newact = act[ act['deviceId'] == deviceid ].copy()
# newdata['act_label'] = newdata.apply(makeStatus, axis=1)
# #     newdata = newdata.append(newact)
    
# # newdata
# newdata

from datetime import datetime


def makeMD(x):
#     print(x['activity_key'])
    ts = datetime.fromtimestamp(x['activity_key']*600)
    return ts.strftime('%m-%d')

act['day'] = act.apply(makeMD, axis=1)
# act['day'] = act['activity_key'].apply(makeMD, axis=1)


act['act_label'] = act.apply(makeStatus, axis=1)

# deviceid, act_label별 집계수를 구한다.
act['new_cnt'] = act.groupby(['day','act_label'])['act_label'].transform('count')
act

,activity_key,avg_presence,avg_people_cnt,avg_activity,max_activity,cnt,first_eventtime,lastStatus,day,act_label,new_cnt
61259,2769933,0,0.0,4.830583,14.8982,6,NaN,NaN,09-01,1,27
61260,2769937,0,0.0,5.206659,22.1080,54,2022-09-01 01:16:03.771,fall_exit,09-01,3,20
61261,2769938,0,0.0,2.848062,16.0897,21,2022-09-01 01:20:08.681,fall_exit,09-01,3,20
61262,2769940,0,0.0,5.868583,34.3912,42,2022-09-01 01:45:46.251,fall_exit,09-01,3,20
61263,2769944,0,0.0,0.090550,0.0942,2,NaN,NaN,09-01,1,27
...,...,...,...,...,...,...,...,...,...,...,...
63513,2774242,0,0.0,8.335829,48.1720,14,NaN,NaN,09-30,2,48
63514,2774243,0,0.0,1.285760,2.6238,5,NaN,NaN,09-30,1,26
63515,2774244,0,0.0,3.931575,14.7158,8,NaN,NaN,09-30,1,26
63516,2774245,0,0.0,4.870035,32.7002,96,2022-09-30 23:10:36.728,fall_exit,09-30,3,15


In [121]:
newdata1 = act[['day', 'act_label', 'new_cnt']]
newdata1


,day,act_label,new_cnt
61259,09-01,1,27
61260,09-01,3,20
61261,09-01,3,20
61262,09-01,3,20
61263,09-01,1,27
...,...,...,...
63513,09-30,2,48
63514,09-30,1,26
63515,09-30,1,26
63516,09-30,3,15


In [122]:
newdata1 = newdata1.drop_duplicates()
newdata1

,day,act_label,new_cnt
61259,09-01,1,27
61260,09-01,3,20
61268,09-01,2,37
61343,09-02,1,26
61345,09-02,3,8
...,...,...,...
63426,09-30,1,26
63427,09-30,3,15
63432,09-30,2,48
63438,09-30,4,2


In [43]:
# #디바이스별로 집계를 구하고, 그 결과를 newdata에 저장함.
# newdata = pd.DataFrame()

# # group의 키 목록을 구함
# grouped = newdata1.groupby('day')

# for deviceid in grouped.groups.keys():
# #     newact = newdata1[ act['deviceId'] == deviceid ].copy()
#     newact['act_label'] = newact.apply(makeStatus, axis=1)
#     newdata = newdata.append(newact)
    
# newdata


# newdata1.groupby(['day', 'act_label'])['new_cnt']

In [123]:
gp = newdata1.groupby('day')

kim = pd.DataFrame(np.zeros((29,3)), columns=['day', 'act_label', 'act_cnt'])

index = 0 

for id in gp.groups.keys():
    kim.loc[index] = [id, 0, np.nan]
    index += 1
    kim.loc[index] = [id, 1, np.nan]
    index += 1
    kim.loc[index] = [id, 2, np.nan]
    index += 1
    kim.loc[index] = [id, 3, np.nan]
    index += 1
    kim.loc[index] = [id, 4, np.nan]
    index += 1

kim.act_cnt = np.nan
kim

,day,act_label,act_cnt
0,09-01,0.0,NaN
1,09-01,1.0,NaN
2,09-01,2.0,NaN
3,09-01,3.0,NaN
4,09-01,4.0,NaN
...,...,...,...
145,09-30,0.0,NaN
146,09-30,1.0,NaN
147,09-30,2.0,NaN
148,09-30,3.0,NaN


In [124]:
kim = pd.merge(kim, newdata1, on=['day', 'act_label'], how="left")
kim

,day,act_label,act_cnt,new_cnt
0,09-01,0.0,NaN,NaN
1,09-01,1.0,NaN,27.0
2,09-01,2.0,NaN,37.0
3,09-01,3.0,NaN,20.0
4,09-01,4.0,NaN,NaN
...,...,...,...,...
145,09-30,0.0,NaN,1.0
146,09-30,1.0,NaN,26.0
147,09-30,2.0,NaN,48.0
148,09-30,3.0,NaN,15.0


In [125]:
kim['new_cnt'] = kim['new_cnt'].fillna(0)
kim

,day,act_label,act_cnt,new_cnt
0,09-01,0.0,NaN,0.0
1,09-01,1.0,NaN,27.0
2,09-01,2.0,NaN,37.0
3,09-01,3.0,NaN,20.0
4,09-01,4.0,NaN,0.0
...,...,...,...,...
145,09-30,0.0,NaN,1.0
146,09-30,1.0,NaN,26.0
147,09-30,2.0,NaN,48.0
148,09-30,3.0,NaN,15.0


In [126]:
kim = kim.drop(['act_cnt'], axis=1)

In [127]:
reshaped = kim.pivot('day', 'act_label', 'new_cnt')
reshaped

C:\Users\sunin\AppData\Local\Temp\ipykernel_39492\1684931186.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  reshaped = kim.pivot('day', 'act_label', 'new_cnt')


act_label,0.0,1.0,2.0,3.0,4.0
day,,,,,
09-01,0.0,27.0,37.0,20.0,0.0
09-02,0.0,26.0,47.0,8.0,0.0
09-03,0.0,34.0,44.0,15.0,0.0
09-04,0.0,17.0,30.0,15.0,0.0
09-05,0.0,24.0,43.0,14.0,0.0
09-06,1.0,22.0,32.0,11.0,0.0
09-07,0.0,32.0,48.0,19.0,0.0
09-08,0.0,25.0,69.0,9.0,0.0
09-09,0.0,13.0,57.0,21.0,0.0


In [70]:
reshaped = reshaped.reset_index()

In [71]:
reshaped

day new_cnt                           
act_label           0.0    1.0     2.0    3.0  4.0
0          0901     0.0  655.0  1141.0  438.0  0.0
1          0902     0.0  655.0  1141.0  438.0  0.0
2          0903     0.0  655.0  1141.0  438.0  0.0
3          0904     0.0  655.0  1141.0  438.0  0.0
4          0905     0.0  655.0  1141.0  438.0  0.0
5          0906    19.0  655.0  1141.0  438.0  0.0
6          0907     0.0  655.0  1141.0  438.0  0.0
7          0908     0.0  655.0  1141.0  438.0  0.0
8          0909     0.0  655.0  1141.0  438.0  0.0
9          0910     0.0  655.0     0.0    0.0  0.0
10         0911    19.0  655.0  1141.0  438.0  0.0
11         0912     0.0  655.0  1141.0  438.0  0.0
12         0913     0.0  655.0  1141.0  438.0  0.0
13         0914    19.0  655.0  1141.0  438.0  6.0
14         0915     0.0  655.0  1141.0  438.0  0.0
15         0916     0.0  655.0  1141.0    0.0  0.0
16         0917     0.0  655.0  1141.0  438.0  0.0
17         0918    19.0  655.0  1141.0  438.0  0.0
18         0919     0.0  655.0  1141.0  438.0  0.0
19         0920     0.0  655.0     0.0  438.0  0.0
20         0921    19.0  655.0  1141.0  438.0  0.0
21         0922    19.0  655.0  1141.0  438.0  0.0
22         0923     0.0  655.0  1141.0  438.0  6.0
23         0924    19.0  655.0  1141.0  438.0  0.0
24         0925    19.0  655.0  1141.0  438.0  6.0
25         0926    19.0  655.0  1141.0  438.0  0.0
26         0927     0.0  655.0  1141.0  438.0  6.0
27         0928    19.0  655.0  1141.0  438.0  0.0
28         0929     0.0  655.0  1141.0  438.0  0.0
29         0930    19.0  655.0  1141.0  438.0  6.0

In [117]:
reshaped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, 0901 to 0930
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0.0     30 non-null     float64
 1   1.0     30 non-null     float64
 2   2.0     30 non-null     float64
 3   3.0     30 non-null     float64
 4   4.0     30 non-null     float64
dtypes: float64(5)
memory usage: 1.4+ KB


In [128]:
reshaped.to_csv('stacked_one_25.csv')